<a href="https://colab.research.google.com/github/AlexeyProvorov/Templates/blob/main/Example%20for%20tf.keras.layers.Layer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пример использования tf.keras.layers.Layer для создания модели

In [12]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



In [16]:
class DenseLayer(tf.keras.layers.Layer):
    def __init__(self, units=1):
        super().__init__()
        self.units = units
        self.rate = 0.01

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer="random_normal",
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        regular = tf.reduce_mean(tf.square(self.w))
        self.add_loss(regular)
        self.add_metric(regular, name="mean square weights")

        return tf.matmul(inputs, self.w) + self.b



class NeuralNetwork(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_1 = DenseLayer(128)
        self.layer_2 = DenseLayer(10)

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = tf.nn.relu(x)
        x = self.layer_2(x)
        x = tf.nn.softmax(x)
        return x

model = NeuralNetwork()


model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])



(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28*28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28*28])

y_train = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)


model.fit(x_train, y_train, batch_size=32, epochs=5)

print(model.evaluate(x_test, y_test_cat) )

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.3054 - accuracy: 0.9194 - mean square weights: 0.0102
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1489 - accuracy: 0.9640 - mean square weights: 0.0169
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1166 - accuracy: 0.9736 - mean square weights: 0.0196
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1018 - accuracy: 0.9785 - mean square weights: 0.0208
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.1094 - accuracy: 0.9761 - mean square weights: 0.0211
[0.10935395210981369, 0.9761000275611877, 0.021107666194438934, 0.021107666194438934]
